In [ ]:
import numpy as np
from roguewave.wavespectra.estimators.mem2 import mem2
from roguewave.wavespectra.estimators.mem import mem
from numba_progress import ProgressBar
from typing import Literal

import os
import pickle

import pandas as pd
import wavespectra 
import xarray as xr

import matplotlib.pyplot as plt

Estimators = Literal["mem", "mem2"]


# -----------------------------------------------------------------------------
#                       Boilerplate Interfaces
# -----------------------------------------------------------------------------
def estimate_directional_spectrum_from_moments(
    e: np.ndarray,
    a1: np.ndarray,
    b1: np.ndarray,
    a2: np.ndarray,
    b2: np.ndarray,
    direction: np.ndarray,
    method: Estimators = "mem2",
    **kwargs,
) -> np.ndarray:
    """
    Construct a 2D directional distribution based on the directional moments and a spectral
    reconstruction method.

    :param number_of_directions: length of the directional vector for the
    2D spectrum. Directions returned are in degrees

    :param method: Choose a method in ['mem','mem2']
        mem: maximum entrophy (in the Boltzmann sense) method
        Lygre, A., & Krogstad, H. E. (1986). Explicit expression and
        fast but tends to create narrow spectra anderroneous secondary peaks.

        mem2: use entrophy (in the Shannon sense) to maximize. Likely
        best method see- Benoit, M. (1993).

    REFERENCES:
    Benoit, M. (1993). Practical comparative performance survey of methods
        used for estimating directional wave spectra from heave-pitch-roll data.
        In Coastal Engineering 1992 (pp. 62-75).

    Lygre, A., & Krogstad, H. E. (1986). Maximum entropy estimation of the
        directional distribution in ocean wave spectra.
        Journal of Physical Oceanography, 16(12), 2052-2060.

    """
    return (
        estimate_directional_distribution(a1, b1, a2, b2, direction, method, **kwargs)
        * e[..., None]
    )


def estimate_directional_distribution(
    a1: np.ndarray,
    b1: np.ndarray,
    a2: np.ndarray,
    b2: np.ndarray,
    direction: np.ndarray,
    method: Estimators = "mem2",
    **kwargs,
) -> np.ndarray:
    """
    Construct a 2D directional distribution based on the directional moments and a spectral
    reconstruction method.

    :param number_of_directions: length of the directional vector for the
    2D spectrum. Directions returned are in degrees

    :param method: Choose a method in ['mem','mem2']
        mem: maximum entrophy (in the Boltzmann sense) method
        Lygre, A., & Krogstad, H. E. (1986). Explicit expression and
        fast but tends to create narrow spectra anderroneous secondary peaks.

        mem2: use entrophy (in the Shannon sense) to maximize. Likely
        best method see- Benoit, M. (1993).

    REFERENCES:
    Benoit, M. (1993). Practical comparative performance survey of methods
        used for estimating directional wave spectra from heave-pitch-roll data.
        In Coastal Engineering 1992 (pp. 62-75).

    Lygre, A., & Krogstad, H. E. (1986). Maximum entropy estimation of the
        directional distribution in ocean wave spectra.
        Journal of Physical Oceanography, 16(12), 2052-2060.

    """

    # Jacobian to transform distribution as function of radian angles into
    # degrees.
    Jacobian = np.pi / 180
    direction_radians = direction * Jacobian

    if method.lower() in ["maximum_entropy_method", "mem"]:
        # reconstruct the directional distribution using the maximum entropy
        # method.
        function = mem
    elif method.lower() in ["maximum_entrophy_method2", "mem2"]:
        function = mem2
    else:
        raise Exception(f"unsupported spectral estimator method: {method}")

    output_shape = list(a1.shape) + [len(direction)]
    if a1.ndim == 1:
        input_shape = [1, a1.shape[-1]]
    else:
        input_shape = [np.prod(a1.shape[0:-1]), a1.shape[-1]]

    a1 = a1.reshape(input_shape)
    b1 = b1.reshape(input_shape)
    a2 = a2.reshape(input_shape)
    b2 = b2.reshape(input_shape)

    number_of_iterations = a1.shape[0]
    if number_of_iterations < 10:
        disable = True
    else:
        disable = False

    if method != "mem2":
        msg = f"Reconstructing 2d spectrum with {method} using implementation: "
    else:
        solution_method = kwargs.get("solution_method", "scipy")
        msg = f"Reconstructing 2d spectrum with {method} using solution_method {solution_method}"

    with ProgressBar(total=number_of_iterations, disable=disable, desc=msg) as progress:
        res = function(direction_radians, a1, b1, a2, b2, progress, **kwargs)

    return res.reshape(output_shape) * Jacobian

In [ ]:
DATA_DIRECTORY = '/vortexfs1/home/csherwood/proj/NOPP/buoy_data/'
DATA_FILENAME = 'hurricane_ian_spotter_data_v1.pickle'

with open(os.path.join(DATA_DIRECTORY, DATA_FILENAME), 'rb') as handle:
    spotter = pickle.load(handle)

# `spotter` is a python dictionary of Pandas DataFrames, keyed by
# each drifter ID. The drifter ids can then be accessed as follows:
spotter_ids = list(spotter.keys())
spotter_id = spotter_ids[0]

# Extract the observation times that contain spectral data.
only_waves = spotter[spotter_id]['energy_density'].notnull()
drifter = spotter[spotter_id][only_waves]

# Exract the coordinate arrays; note that the frequency array is
# uniform across the Spotter observations, so we can just 
# use the array in the first index of the DataFrame.
time = drifter.index.to_numpy()
freq = drifter['frequency'][0] 

# Extract the variable arrays.
efth = np.stack(drifter['energy_density'])
lat = drifter['latitude']
lon = drifter['longitude']

# Construct the dataset. This must match the conventions used by the
# wavespectra package:
# (https://wavespectra.readthedocs.io/en/latest/conventions.html#)
# The directional spectrum is computed using the
# directional moments and an estimator (e.g. MEM).
directions = np.arange(0., 360., 10)
efth_array = np.zeros( (len(time), 1, 1, len(freq), len(directions)))
for i in range(len(time)):
    #print(np.sum(drifter.a1[i]))
    ea = estimate_directional_distribution( drifter.a1[i], drifter.b1[i], \
                                                     drifter.a2[i], drifter.b2[i], directions, method='mem2' )
    #print(np.shape(ea))
    efth_array[i,0,0,:,:] = ea
    #print( np.sum(ea) )

# efth(time, freq) and not efth(time, freq, dir).
ds = xr.Dataset(
    data_vars=dict(
        efth=(["time", "lat", "lon", "freq", "dir"], efth_array),
        # efth=(["time", "freq"], efth), # Non-directinal...delete this line if using above (Note: efth may need to be transposed)
        site="",
    ),
    coords=dict(
        time=time,
        lat=0,
        lon=0,
        freq=freq,
        dir=directions, #TODO: need to compute this from directional moments
    ),
    attrs=dict(
        # Attributes here; wavespectra would put significant wave
        # height, etc., here but these are a function of time so it
        # might be reasonable to set them as data_vars instead (as 
        # a function of the time coordinate)
    )
)

In [ ]:
ds

In [ ]:
ds.efth.spec.hs()

In [ ]:
efth_array

In [ ]:
directions = np.arange(0., 360., 10)
efth_array = np.zeros( (len(time), len(freq), len(directions)))
print(np.shape(efth_array))

In [ ]:
for i in range(len(time)):
   efth_array[i,:,:] = estimate_directional_distribution( drifter.a1[i], drifter.b1[i], \
                                                         drifter.a2[i], drifter.b2[i], directions, method='mem' )

In [ ]:
np.shape(efth_array)
efth_array

In [ ]:
plt.pcolormesh(np.squeeze(efth_array[20]))